In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,High School Musical,All Or Nothing,"They can play ""let's pretend"" \r\nMake 'em li..."
1,Bon Jovi,I Don't Want To Live Forever,Frank Sinatra died today \r\nI came home and ...
2,Alabama,Here We Are,"Here we are, once again \r\nStronger now than..."
3,Michael W. Smith,"Sing Noel, Sing Hallelujah",Come \r\nBehold this child \r\nIn the manger...
4,Tim McGraw,The One That Got Away,You got your head in your supper \r\nAnd the ...
5,Electric Light Orchestra,Lucky Man,I'm very lucky I think you'll agree \r\n \r\...
6,Cinderella,In From The Outside,I've been loved and I've been shoved \r\nbeat...
7,Hank Williams Jr.,That's How I Wanted It To Be,I thought I'd like to leave the black side of ...
8,Bob Dylan,Big Yellow Taxi,"They pave paradise, they put up a parking lot ..."
9,Perry Como,If,"If they made me a king, I'd be but a slave to ..."


In [9]:
df['text'][0]

'They can play "let\'s pretend"  \r\nMake \'em like them my best friend  \r\nTelling their lies  \r\nAnd they can scheme behind my back  \r\nMake fiction not a fact  \r\nTry to re-write my life  \r\n  \r\nBut they ain\'t got a clue  \r\nIf I were in their shoes  \r\nI\'d shut right up  \r\n  \r\nTake me as I am  \r\nOr not at all  \r\nI ain\'t ever gonna change myself for no one  \r\nNo compromise  \r\n\'Cause it\'s my life  \r\nIt\'s all or nothing.  \r\n  \r\nThey can laugh all they like  \r\nGot no need to justify  \r\nI am who I am  \r\nAnd they can do their worst if they dare  \r\nIf it makes them happy we don\'t care  \r\nIt\'s out of their hands  \r\n  \r\n\'Cause they ain\'t got a clue  \r\nIf we were in their shoes  \r\nWe\'d shut right up  \r\n  \r\nTake us as we are  \r\nOr not at all  \r\nWe ain\'t ever gonna change ourselves for no one  \r\nNo compromise \'cause it\'s our life  \r\nIt\'s all or nothing.  \r\n  \r\nTake me as I am  \r\nOr not at all  \r\nI ain\'t ever gonna

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.01017109, 0.03671516, ..., 0.03456287, 0.04597371,
       0.06759877])

In [18]:
def recommendation(song_df):
    # Check if any rows match the condition
    if not df[df['song'] == song_df].empty:
        idx = df[df['song'] == song_df].index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
        
        songs = []
        for m_id in distances[1:21]:
            songs.append(df.iloc[m_id[0]].song)
            
        return songs
    
    else:
        # Handle the case when no matching rows are found
        print(f"No data found for song: {song_df}")

In [19]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))